# Klasyfikacja probabilistyczna

In [1]:
library(tidyverse)
options(jupyter.rich_display=FALSE,
        repr.plot.width=15,
        repr.plot.height=8)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Naiwny klasyfikator Bayesa

Zapoznaj się z materiałami dostępnymi w [Przewodniku po pakiecie R](http://pbiecek.github.io/Przewodnik/Predykcja/naiwny_bayes.html) oraz [RPubs](https://rpubs.com/mmazurek/322903) oraz wykonaj samodzielnie w RStudio omawiane w nich przykłady. Następnie wykorzystaj naiwny klasyfikator Bayesa do rozwiązania problemu klasyfikacji wiadomości e-mail.

1. Zaimportuj (`read_csv`) i wyświetl (funkcja `head`) dane z pliku `email.csv`

In [3]:
emails = read_csv("data/email.csv")

Rows: 1681 Columns: 1103
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr    (1): message_label
dbl (1102): message_index, ability, abuse, accept, acceptance, accepted, acc...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
head(emails)

  message_index message_label ability abuse accept acceptance accepted access
1 12            ham           0       0     0      0          0        0     
2 21            ham           0       0     0      0          0        0     
3 29            ham           0       0     0      0          0        0     
4 43            ham           0       0     0      0          0        0     
5 59            ham           0       0     0      0          0        0     
6 68            ham           0       0     0      0          0        0     
  account accounting ⋯ worldwide worth write writing www xls xp yahoo yesterday
1 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
2 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
3 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
4 0       0          ⋯ 0         0     0     0       0   0   0  0     0        
5 0       0          ⋯ 0         0     0     0       0

In [8]:
glimpse(emails)

Rows: 1,681
Columns: 1,103
$ message_index   <dbl> 12, 21, 29, 43, 59, 68, 72, 104, 105, 110, 112, 126, 1…
$ message_label   <chr> "ham", "ham", "ham", "ham", "ham", "ham", "ham", "ham"…
$ ability         <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ abuse           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ accept          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ acceptance      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ accepted        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, …
$ access          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ account         <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ accounting      <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ accounts        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ accuracy        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, …
$ achieve    

In [6]:
dim(emails)

[1] 1681 1103

2. Przekształć zmienną `message_label` do typu kategorialnego.

In [9]:
emails$message_label

   [1] "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham" 
  [11] "ham"  "ham"  "ham"  "ham"  "spam" "spam" "spam" "ham"  "spam" "ham" 
  [21] "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "spam" "ham"  "spam" "ham" 
  [31] "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "spam" "ham" 
  [41] "ham"  "ham"  "ham"  "spam" "ham"  "ham"  "spam" "ham"  "ham"  "ham" 
  [51] "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "spam"
  [61] "spam" "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "spam" "ham" 
  [71] "spam" "ham"  "ham"  "spam" "ham"  "ham"  "ham"  "spam" "spam" "ham" 
  [81] "ham"  "ham"  "ham"  "spam" "ham"  "ham"  "ham"  "ham"  "ham"  "ham" 
  [91] "ham"  "ham"  "ham"  "ham"  "spam" "ham"  "ham"  "ham"  "ham"  "ham" 
 [101] "ham"  "spam" "ham"  "ham"  "ham"  "ham"  "spam" "ham"  "ham"  "ham" 
 [111] "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "ham"  "spam"
 [121] "ham"  "ham"  "spam" "spam" "ham"  "spam" "ham"  "spam" "spam" "ham" 

3. Za pomocą funkcji `gather` przekształć zbiór danych tak, aby zamiast kolumny z częstością dla każdego wyrazu mieć dwie kolumny: jedną z wyrazem, drugą z częstością

In [ ]:
email %>%
  gather(word, count, -message_index, -message_label)

4. Pogrupuj dane według `word`, sumując zmienną `count` (sumę nazwij `occurence`) i posortuj wynik w malejącej kolejności `occurence`. Użyj funkcji `slice` do wyświetlenia tylko 10 wyrazów o najwyższej wartości `occurence`.

5. W podobny sposób sprawdź, jakie 10 wyrazów pojawia się w zwykłych wiadomościach, a jakie w wiadomościach spam.

6. Podziel dane na zbiory treningowy i testowy, stosując proporcję 75:25. Następnie wyświetl rozkłady klas dla wszystkich zbiorów danych (`prop.table`).

7. Zbuduj naiwny model Bayesa za pomocą funkcji `naiveBayes`.

In [ ]:
library(e1071)

email_mod <-
  naiveBayes(message_label ~ . - message_index,
             data = email_train,
             laplace = 1)

8. Oceń jak dobrze model radzi sobie z przewidywaniem, czy email w danych testowych to spam, czy zwykła wiadomość (funkcja `predict`, a następnie obliczenie dokładności predykcji modelu w oparciu o wartości macierzy pomyłek).

9. Odpowiedz na pytanie: w jaki sposób moglibyśmy zwiększyć dokładność predykcji?

## Sieć Bayesa

Jednym z najpopularniejszych pakietów R dla sieci Bayesa jest `bnlearn`. Zawiera on wiele różnych algorytmów uczenia SB i wnioskowania oraz zestawy danych.

### Przykład

Jako przykład wykorzystamy zbiór `coronary` z pakietu `bnlearn`, zawierający prawdopodobne czynniki ryzyka zakrzepicy. Zmienne:

- Smoking - palenie (poziomy: 'no', 'yes')
- M. Work - wytężona praca umysłowa (poziomy: 'no', 'yes')
- P. Work - wytężona praca fizyczna (poziomy: 'no', 'yes')
- Pressure - ciśnienie skurczowe krwi (poziomy: '<140', '>140')
- Proteins - stosunek lipoprotein alfa i beta (poziomy: '<3', '>3')
- Family - występowanie choroby niedokrwiennej w rodzinie (poziomy: 'neg', 'pos')

In [ ]:
# załadowanie danych
library(bnlearn)
data(coronary)

Utworzenie i wizualizacja sieci Bayesa

In [ ]:
bn_df <- data.frame(coronary)
res <- hc(bn_df)
plot(res)

Nie wszystkie powstałe powiązania mają sens (np. `Family` jako warunek `M. Work`). Modyfikacja struktury sieci:

In [ ]:
res$arcs <- res$arcs[-which((res$arcs[,'from'] == "M..Work" & 
                               res$arcs[,'to'] == "Family")),]

Następnym krokiem po zapoznaniu się ze strukturą sieci, jest znalezienie tablic prawdopodobieństwa warunkowego (CPT) w każdym węźle. Służy do tego funkcja `bn.fit`. Następnie za pomocą funkcji `print` można podejrzeć co znajduje się w węźle `Protein`:

In [ ]:
fittedbn <- bn.fit(res, data = bn_df)
print(fittedbn$Proteins)

Tak przygotowaną sieć możemy wykorzystać do wnioskowania. Na przykład móżemy znaleźć odpowiedź na pytanie: Jakie jest prawdopodobieństwo, że osoba niepaląca z ciśnieniem >140, będzie miała poziom białek <3?

In [ ]:
cpquery(fittedbn, event = (Proteins=="<3"), evidence = ((Smoking=="no") & 
                                                          (Pressure == ">140")))